# CIAN.ru

Загружаем все необходимые пакеты.

In [ ]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd
import time

Здест мы будем писать все пользовательские функции, которые нам понадобятся для парсинга отдельных элементов в страничке. Описывать буду в комментариях

In [ ]:
#Функция из лекции
def html_stripper(text):
    return re.sub('<[^<]+?>', '', str(text))
#Эта функция возвращает количество времени от метро на машине или пешком
def getMetrodist(flat_page):
    metro = flat_page.find('span', attrs={'class':'object_item_metro_comment'}) #ищем нужную информацию в классе object_item_metro_comment
    metro = re.split('<span>|\n|', str(metro))
    for i in metro:
        if i.strip().isdigit():
            return int(i)
    return 0
#Эта функция возвращает 1 если идем пешком от метро и 0 если на транспорте
def getWalk(flat_page):
    metro = flat_page.find('span', attrs={'class':'object_item_metro_comment'})
    metro = re.split('<span>|\n|', str(metro))
    for i in metro:
        if i.strip()=='пешком':
            return 1
    return 0
#Функция из лекции
def getPrice(flat_page):
    price = flat_page.find('div', attrs={'class':'object_descr_price'})
    price = re.split('<div>|руб|\W', str(price))
    price = "".join([i for i in price if i.isdigit()][-3:])
    return int(price)
#Функция из лекции
def getCoords(flat_page):
    coords = flat_page.find('div', attrs={'class':'map_info_button_extend'}).contents[1]
    coords = re.split('&amp|center=|%2C', str(coords))
    coords_list = []
    for item in coords:
        if item[0].isdigit():
            coords_list.append(item)
    lat = float(coords_list[0])
    lon = float(coords_list[1])
    return lat, lon
#Функция из лекции
def getRoom(flat_page):
    rooms = flat_page.find('div', attrs={'class':'object_descr_title'})
    rooms = html_stripper(rooms)
    room_number = ''
    for i in re.split('-|\n', rooms):
        if 'комн' in i:
            break
        else:
            room_number += i
    room_number = "".join(room_number.split())
    return room_number
#Функция которая возвращает 1 если дом кирпичный и т.д. и 0 если другое
def getBrick(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})#Информацию ищем в таблице
    table = html_stripper(table)
    Bricks=re.split('Тип дома|Тип продажи', table)[1] #эта информация лежит между Тип дома и тип продажи
    Brick=re.split('\n|:|,', Bricks)
    for i in Brick:
        if i.strip() in ['кирпичный','монолит','жб','монолитный', 'кирпично-монолитный']: 
            return 1
        elif i.strip()=='другой':
            return 0
    return 0
#Функция возвращает 1 если дом новый и 0 если вторичка. Так же информация лежит в таблице между тип дома и тип продажи
def getNew(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    News=re.split('Тип дома|Тип продажи', table)[1]
    New=re.split('\n|:|,', News)
    for i in New:
        if i.strip()=='первичный рынок':
            return 1
        elif i.strip()=='вторичка':
            return 0
    return 0
#Функция возвращает сразу на каком этаже наша квартира и сколько этажей в доме. если в таблице только одно значение то записываем его в Floor, а NFloor указываем 0
def getFloor(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    Floors=re.split('Этаж|Тип дома', table)[1]#Информация лежит между Этаж и Тип дома
    floors=re.split('\n|\W', Floors) #\W убирает все символы и оставляет только числа
    tmp=[0,0]
    count=0
    for i in floors:
        if i.strip().isdigit():
            tmp[count]=int(i)
            count=count+1
    return tmp[0],tmp[1]
#Функция возвращает 1 если балкон есть и 0 если балкона нет. Посмотрел разные варианты как люди записывают что и них есть балкон. Получил нижний список.
def getBal(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    Bals=re.split('Балкон|Лифт', table)[1]
    Bal=re.split('\n\w', Bals)
    for i in Bal:
        if i.strip() in ['да', 'лоджия', 'лодж.','балк.','есть']:
            return 1
    return 0
#Функция возвращает 1 если телефон есть 0 если нет. Такая же проблема здесь. Разные люди пишут по разному.
def getTel(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    Tels=re.split('Телефон|Вид из окна', table)[1]
    Tel=re.split('\n|\w', Tels)
    for i in Tel:
        if i.strip()  in ['да', 'есть']:
            return 1
    return 0
#Функция возвращает какая общая площадь квартиры. Пришлось писать дополнительную функцию get_num чтоб обрабатывать такие случаи как "87,7 м2". 
def getTotsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    total_areas=re.split('Общая площадь:|Площадь комнат:', table)[1]
    total_area = 0
    for i in re.split('\n|м2', total_areas):
        if i.strip()!='':
            total_area=get_num(i)
        if total_area>0:
            break
    return total_area
#Жилая площадь. Тоже самое. Иногда не было информации по квадратуре, тогда записывал как 0
def getLivesp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    Livesps=re.split('Жилая площадь:|Площадь кухни:', table)[1]
    Livesp = 0
    for i in re.split('\n|м2', Livesps):
        if i.strip()!='':
            Livesp=get_num(i)
        if Livesp>0:
            break
    return Livesp
#Площадб кухни. Также все
def getKitsp(flat_page):
    table = flat_page.find('table', attrs = {'class':'object_descr_props'})
    table = html_stripper(table)
    Kitsps=re.split('Площадь кухни:|:', table)[1]
    Kitsp =0
    for i in re.split('\n|м2', Kitsps):
        if i.strip()!='':
            Kitsp=get_num(i)
        if Kitsp>0:
            break
    return(Kitsp)
#Функция ищет в строке числа и переводит их в десятичную запись. если числа нет то возвращает 0
def get_num(x):
    x=x.replace(',','.')
    tmp=''.join(ele for ele in x if ele.isdigit() or ele == '.')
    if tmp!='':
        return float(tmp)
    return 0

In [ ]:
#записываем все адреса округов в один лист
district=['','','','','','','','','']
#ЦАО
district[0] = 'http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=13&district%5B1%5D=14&district%5B2%5D=15&district%5B3%5D=16&district%5B4%5D=17&district%5B5%5D=18&district%5B6%5D=19&district%5B7%5D=20&district%5B8%5D=21&district%5B9%5D=22&engine_version=2&offer_type=flat&p={}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#САО
district[1]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=23&district%5B10%5D=33&district%5B11%5D=34&district%5B12%5D=35&district%5B13%5D=36&district%5B14%5D=37&district%5B15%5D=38&district%5B1%5D=24&district%5B2%5D=25&district%5B3%5D=26&district%5B4%5D=27&district%5B5%5D=28&district%5B6%5D=29&district%5B7%5D=30&district%5B8%5D=31&district%5B9%5D=32&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#СВАО
district[2]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=39&district%5B10%5D=49&district%5B11%5D=50&district%5B12%5D=51&district%5B13%5D=52&district%5B14%5D=53&district%5B15%5D=54&district%5B16%5D=55&district%5B1%5D=40&district%5B2%5D=41&district%5B3%5D=42&district%5B4%5D=43&district%5B5%5D=44&district%5B6%5D=45&district%5B7%5D=46&district%5B8%5D=47&district%5B9%5D=48&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ВАО
district[3]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=56&district%5B10%5D=66&district%5B11%5D=67&district%5B12%5D=68&district%5B13%5D=69&district%5B14%5D=70&district%5B15%5D=71&district%5B1%5D=57&district%5B2%5D=58&district%5B3%5D=59&district%5B4%5D=60&district%5B5%5D=61&district%5B6%5D=62&district%5B7%5D=63&district%5B8%5D=64&district%5B9%5D=65&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЮВАО
district[4]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=72&district%5B10%5D=82&district%5B11%5D=83&district%5B1%5D=73&district%5B2%5D=74&district%5B3%5D=75&district%5B4%5D=76&district%5B5%5D=77&district%5B6%5D=78&district%5B7%5D=79&district%5B8%5D=80&district%5B9%5D=81&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЮАО
district[5]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=84&district%5B10%5D=94&district%5B11%5D=95&district%5B12%5D=96&district%5B13%5D=97&district%5B14%5D=98&district%5B15%5D=99&district%5B1%5D=85&district%5B2%5D=86&district%5B3%5D=87&district%5B4%5D=88&district%5B5%5D=89&district%5B6%5D=90&district%5B7%5D=91&district%5B8%5D=92&district%5B9%5D=93&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЮЗАО
district[6]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=100&district%5B10%5D=110&district%5B11%5D=111&district%5B1%5D=101&district%5B2%5D=102&district%5B3%5D=103&district%5B4%5D=104&district%5B5%5D=105&district%5B6%5D=106&district%5B7%5D=107&district%5B8%5D=108&district%5B9%5D=109&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#ЗАО
district[7]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=112&district%5B10%5D=122&district%5B11%5D=123&district%5B12%5D=124&district%5B13%5D=348&district%5B14%5D=349&district%5B15%5D=350&district%5B1%5D=113&district%5B2%5D=114&district%5B3%5D=115&district%5B4%5D=116&district%5B5%5D=117&district%5B6%5D=118&district%5B7%5D=119&district%5B8%5D=120&district%5B9%5D=121&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'
#СЗАО
district[8]='http://www.cian.ru/cat.php?deal_type=sale&district%5B0%5D=125&district%5B1%5D=126&district%5B2%5D=127&district%5B3%5D=128&district%5B4%5D=129&district%5B5%5D=130&district%5B6%5D=131&district%5B7%5D=132&engine_version=2&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1'

In [ ]:
flat=[]#здесь будем хранить информацию про квартиры
for k in range(9):#пробегаем по всем округам
    page = 1
    links = []
    for page in range(1, 30):
        page_url =  district[k].format(page)

        search_page = requests.get(page_url)
        search_page = search_page.content
        search_page = BeautifulSoup(search_page, 'lxml')

        flat_urls = search_page.findAll('div', attrs = {'ng-class':"{'serp-item_removed': offer.remove.state, 'serp-item_popup-opened': isPopupOpen}"})
        flat_urls = re.split('http://www.cian.ru/sale/flat/|/" ng-class="', str(flat_urls))

        for link in flat_urls:
            if link.isdigit():
                links.append(link)
                
    for i in range(len(links)): #пробегаем по всем квартирам из ссылки
        flat_url = 'http://www.cian.ru/sale/flat/' + str(links[i]) + '/'
        flat_page = requests.get(flat_url)
        flat_page = flat_page.content
        flat_page = BeautifulSoup(flat_page, 'lxml')
        flatStats = {'District':k} #Записываем номер округа
        flatStats['N']=int("".join([x for x in links[i] if x.isdigit()][:])) #Записываем номер квартиры в базе
        flatStats['Walk'] = getWalk(flat_page)
        flatStats['Metrodist'] = getMetrodist(flat_page)
        flatStats['Price'] = getPrice(flat_page)
        flatStats['lat'], flatStats['lon'] = getCoords(flat_page)
        flatStats['rooms'] = getRoom(flat_page)
        flatStats['Brick']=getBrick(flat_page)
        flatStats['New']=getNew(flat_page)
        flatStats['Floor'],flatStats['NFloor']=getFloor(flat_page)
        flatStats['Tel'] = getTel(flat_page)
        flatStats['Bal'] = getBal(flat_page)
        flatStats['Totsp'] = getTotsp(flat_page)
        flatStats['Livesp'] = getLivesp(flat_page)
        flatStats['Kitsp'] = getKitsp(flat_page)
        flat.append(flatStats) #добавляем квартиры в flat

In [ ]:
#Записываем все полученные данные в csv файл.
import csv
keys = flat[0].keys()
with open('flat.csv', 'wb') as output_file:
    dict_writer = csv.DictWriter(output_file, keys)
    dict_writer.writeheader()
    dict_writer.writerows(flat)